# Urban Sounds Classification using Deep Learning

## Udacity-MLND

### Multiple_Feature_MLP

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
#importing Audio Visualisation Libraries
from librosa import display
import librosa
#Getting the csv as dataframe using pandas
data=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")

In [2]:
#displaying the structure of the CSV
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [3]:
#preprocessing using all features set
x_train=[]
x_test=[]
y_train=[]
y_test=[]
path="UrbanSound8K/audio/fold"
for i in tqdm(range(len(data))):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["slice_file_name"]
    label=data.iloc[i]["classID"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(40,5))
    if(fold_no!='10'):
      x_train.append(features)
      y_train.append(label)
    else:
      x_test.append(features)
      y_test.append(label)

  1%|▌                                                                             | 67/8732 [00:30<1:06:07,  2.18it/s]C:\Anaconda3\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
100%|████████████████████████████████████████████████████████████████████████████| 8732/8732 [1:04:30<00:00,  2.83it/s]


In [4]:
#converting the lists into numpy arrays
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7895, 40, 5), (837, 40, 5), (7895,), (837,))

In [5]:
#reshaping into 2d to save in csv format
x_train_2d=np.reshape(x_train,(x_train.shape[0],x_train.shape[1]*x_train.shape[2]))
x_test_2d=np.reshape(x_test,(x_test.shape[0],x_test.shape[1]*x_test.shape[2]))
x_train_2d.shape,x_test_2d.shape

((7895, 200), (837, 200))

In [6]:
#saving the data numpy arrays
np.savetxt("train_data_multi.csv", x_train_2d, delimiter=",")
np.savetxt("test_data_multi.csv",x_test_2d,delimiter=",")
np.savetxt("train_labels_multi.csv",y_train,delimiter=",")
np.savetxt("test_labels_multi.csv",y_test,delimiter=",")

In [2]:
#extracting data from csv files into numpy arrays
from numpy import genfromtxt
x_train = genfromtxt('train_data_multi.csv', delimiter=',')
y_train = genfromtxt('train_labels_multi.csv', delimiter=',')
x_test = genfromtxt('test_data_multi.csv', delimiter=',')
y_test = genfromtxt('test_labels_multi.csv', delimiter=',')

In [3]:
#shape
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7895, 200), (837, 200), (7895,), (837,))

In [4]:
#converting to one hot
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
y_train.shape,y_test.shape

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

((7895, 10), (837, 10))

In [19]:
#shapes
x_train.shape,x_test.shape

((7895, 200), (837, 200))

In [5]:
from keras import Sequential
from keras.layers import Dense,Dropout,Activation

In [7]:
#forming model
model=Sequential()
#building the model
model.add(Dense(units=256,activation='relu',input_dim=200))
model.add(Dropout(0.5))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10,activation='softmax'))

In [8]:
#compiling
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

W0831 20:59:28.733877 16832 deprecation_wrapper.py:119] From C:\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0831 20:59:28.775738 16832 deprecation_wrapper.py:119] From C:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [11]:
from keras.callbacks import ModelCheckpoint 
#fitting
model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),batch_size=50,callbacks=[ModelCheckpoint(filepath='Saved_models/multi_mlp.hdf5', verbose=1, save_best_only=True)], verbose=1)

Train on 7895 samples, validate on 837 samples
Epoch 1/50
7895/7895 [==============================] - ETA: 1s - loss: 0.5773 - acc: 0.860 - ETA: 1s - loss: 0.5725 - acc: 0.822 - ETA: 1s - loss: 0.5971 - acc: 0.803 - ETA: 0s - loss: 0.6054 - acc: 0.800 - ETA: 0s - loss: 0.6068 - acc: 0.802 - ETA: 0s - loss: 0.6165 - acc: 0.804 - ETA: 0s - loss: 0.6098 - acc: 0.804 - ETA: 0s - loss: 0.6170 - acc: 0.801 - ETA: 0s - loss: 0.6124 - acc: 0.804 - ETA: 0s - loss: 0.6042 - acc: 0.806 - ETA: 0s - loss: 0.6085 - acc: 0.805 - ETA: 0s - loss: 0.6086 - acc: 0.804 - ETA: 0s - loss: 0.6107 - acc: 0.803 - ETA: 0s - loss: 0.6173 - acc: 0.801 - ETA: 0s - loss: 0.6159 - acc: 0.800 - ETA: 0s - loss: 0.6180 - acc: 0.800 - ETA: 0s - loss: 0.6159 - acc: 0.801 - ETA: 0s - loss: 0.6128 - acc: 0.802 - ETA: 0s - loss: 0.6137 - acc: 0.802 - ETA: 0s - loss: 0.6096 - acc: 0.802 - ETA: 0s - loss: 0.6125 - acc: 0.801 - 1s 147us/step - loss: 0.6178 - acc: 0.8000 - val_loss: 1.2023 - val_acc: 0.6213

Epoch 00001: val_l

7895/7895 [==============================] - ETA: 1s - loss: 0.5462 - acc: 0.840 - ETA: 1s - loss: 0.6043 - acc: 0.802 - ETA: 1s - loss: 0.6509 - acc: 0.797 - ETA: 0s - loss: 0.6035 - acc: 0.810 - ETA: 0s - loss: 0.5831 - acc: 0.812 - ETA: 0s - loss: 0.5705 - acc: 0.813 - ETA: 0s - loss: 0.5699 - acc: 0.813 - ETA: 0s - loss: 0.5601 - acc: 0.816 - ETA: 0s - loss: 0.5640 - acc: 0.814 - ETA: 0s - loss: 0.5771 - acc: 0.811 - ETA: 0s - loss: 0.5819 - acc: 0.807 - ETA: 0s - loss: 0.5837 - acc: 0.808 - ETA: 0s - loss: 0.5877 - acc: 0.808 - ETA: 0s - loss: 0.5881 - acc: 0.807 - ETA: 0s - loss: 0.5881 - acc: 0.809 - ETA: 0s - loss: 0.5856 - acc: 0.812 - ETA: 0s - loss: 0.5764 - acc: 0.814 - ETA: 0s - loss: 0.5811 - acc: 0.813 - ETA: 0s - loss: 0.5825 - acc: 0.812 - ETA: 0s - loss: 0.5796 - acc: 0.813 - ETA: 0s - loss: 0.5836 - acc: 0.813 - ETA: 0s - loss: 0.5813 - acc: 0.813 - ETA: 0s - loss: 0.5834 - acc: 0.812 - 1s 159us/step - loss: 0.5862 - acc: 0.8115 - val_loss: 1.2440 - val_acc: 0.6260



7895/7895 [==============================] - ETA: 1s - loss: 0.5910 - acc: 0.840 - ETA: 0s - loss: 0.5345 - acc: 0.822 - ETA: 0s - loss: 0.5315 - acc: 0.827 - ETA: 0s - loss: 0.5564 - acc: 0.816 - ETA: 0s - loss: 0.5628 - acc: 0.813 - ETA: 0s - loss: 0.5551 - acc: 0.817 - ETA: 0s - loss: 0.5388 - acc: 0.822 - ETA: 0s - loss: 0.5331 - acc: 0.823 - ETA: 0s - loss: 0.5358 - acc: 0.821 - ETA: 0s - loss: 0.5431 - acc: 0.820 - ETA: 0s - loss: 0.5349 - acc: 0.822 - ETA: 0s - loss: 0.5336 - acc: 0.824 - ETA: 0s - loss: 0.5371 - acc: 0.826 - ETA: 0s - loss: 0.5391 - acc: 0.827 - ETA: 0s - loss: 0.5431 - acc: 0.827 - ETA: 0s - loss: 0.5376 - acc: 0.828 - ETA: 0s - loss: 0.5359 - acc: 0.829 - ETA: 0s - loss: 0.5381 - acc: 0.828 - ETA: 0s - loss: 0.5396 - acc: 0.828 - ETA: 0s - loss: 0.5377 - acc: 0.828 - ETA: 0s - loss: 0.5374 - acc: 0.828 - ETA: 0s - loss: 0.5339 - acc: 0.829 - 1s 150us/step - loss: 0.5331 - acc: 0.8301 - val_loss: 1.2079 - val_acc: 0.6595

Epoch 00016: val_loss did not improve 

Epoch 00023: val_loss did not improve from 1.16043
Epoch 24/50
7895/7895 [==============================] - ETA: 1s - loss: 0.3144 - acc: 0.900 - ETA: 1s - loss: 0.4309 - acc: 0.860 - ETA: 1s - loss: 0.5125 - acc: 0.832 - ETA: 1s - loss: 0.4907 - acc: 0.836 - ETA: 0s - loss: 0.4924 - acc: 0.831 - ETA: 0s - loss: 0.4946 - acc: 0.833 - ETA: 0s - loss: 0.4842 - acc: 0.837 - ETA: 0s - loss: 0.4768 - acc: 0.840 - ETA: 0s - loss: 0.4761 - acc: 0.839 - ETA: 0s - loss: 0.4751 - acc: 0.840 - ETA: 0s - loss: 0.4717 - acc: 0.843 - ETA: 0s - loss: 0.4682 - acc: 0.846 - ETA: 0s - loss: 0.4757 - acc: 0.846 - ETA: 0s - loss: 0.4766 - acc: 0.845 - ETA: 0s - loss: 0.4761 - acc: 0.845 - ETA: 0s - loss: 0.4763 - acc: 0.846 - ETA: 0s - loss: 0.4733 - acc: 0.847 - ETA: 0s - loss: 0.4788 - acc: 0.845 - ETA: 0s - loss: 0.4744 - acc: 0.846 - ETA: 0s - loss: 0.4742 - acc: 0.847 - ETA: 0s - loss: 0.4709 - acc: 0.847 - ETA: 0s - loss: 0.4795 - acc: 0.846 - 1s 152us/step - loss: 0.4777 - acc: 0.8465 - val_loss: 1


Epoch 00031: val_loss did not improve from 1.16043
Epoch 32/50
7895/7895 [==============================] - ETA: 1s - loss: 0.5514 - acc: 0.720 - ETA: 0s - loss: 0.4945 - acc: 0.824 - ETA: 0s - loss: 0.4538 - acc: 0.853 - ETA: 0s - loss: 0.4670 - acc: 0.856 - ETA: 0s - loss: 0.4568 - acc: 0.858 - ETA: 0s - loss: 0.4602 - acc: 0.855 - ETA: 0s - loss: 0.4567 - acc: 0.854 - ETA: 0s - loss: 0.4578 - acc: 0.852 - ETA: 0s - loss: 0.4624 - acc: 0.851 - ETA: 0s - loss: 0.4701 - acc: 0.849 - ETA: 0s - loss: 0.4637 - acc: 0.850 - ETA: 0s - loss: 0.4683 - acc: 0.850 - ETA: 0s - loss: 0.4616 - acc: 0.851 - ETA: 0s - loss: 0.4609 - acc: 0.851 - ETA: 0s - loss: 0.4578 - acc: 0.853 - ETA: 0s - loss: 0.4658 - acc: 0.850 - ETA: 0s - loss: 0.4714 - acc: 0.848 - ETA: 0s - loss: 0.4756 - acc: 0.847 - ETA: 0s - loss: 0.4737 - acc: 0.849 - ETA: 0s - loss: 0.4733 - acc: 0.849 - ETA: 0s - loss: 0.4717 - acc: 0.849 - 1s 145us/step - loss: 0.4740 - acc: 0.8481 - val_loss: 1.2291 - val_acc: 0.6595

Epoch 00032:

7895/7895 [==============================] - ETA: 1s - loss: 0.4182 - acc: 0.860 - ETA: 1s - loss: 0.4243 - acc: 0.846 - ETA: 0s - loss: 0.3965 - acc: 0.863 - ETA: 0s - loss: 0.3997 - acc: 0.866 - ETA: 0s - loss: 0.4053 - acc: 0.865 - ETA: 0s - loss: 0.4202 - acc: 0.859 - ETA: 0s - loss: 0.4269 - acc: 0.864 - ETA: 0s - loss: 0.4212 - acc: 0.865 - ETA: 0s - loss: 0.4217 - acc: 0.863 - ETA: 0s - loss: 0.4231 - acc: 0.862 - ETA: 0s - loss: 0.4253 - acc: 0.863 - ETA: 0s - loss: 0.4295 - acc: 0.862 - ETA: 0s - loss: 0.4388 - acc: 0.860 - ETA: 0s - loss: 0.4335 - acc: 0.861 - ETA: 0s - loss: 0.4334 - acc: 0.862 - ETA: 0s - loss: 0.4343 - acc: 0.863 - ETA: 0s - loss: 0.4338 - acc: 0.863 - ETA: 0s - loss: 0.4335 - acc: 0.863 - ETA: 0s - loss: 0.4351 - acc: 0.863 - ETA: 0s - loss: 0.4314 - acc: 0.864 - ETA: 0s - loss: 0.4300 - acc: 0.864 - 1s 145us/step - loss: 0.4329 - acc: 0.8632 - val_loss: 1.2049 - val_acc: 0.6691

Epoch 00040: val_loss did not improve from 1.16043
Epoch 41/50
7895/7895 [==

7895/7895 [==============================] - ETA: 1s - loss: 0.3977 - acc: 0.860 - ETA: 1s - loss: 0.4779 - acc: 0.860 - ETA: 1s - loss: 0.4556 - acc: 0.865 - ETA: 0s - loss: 0.4475 - acc: 0.863 - ETA: 0s - loss: 0.4492 - acc: 0.861 - ETA: 0s - loss: 0.4464 - acc: 0.860 - ETA: 0s - loss: 0.4418 - acc: 0.858 - ETA: 0s - loss: 0.4421 - acc: 0.856 - ETA: 0s - loss: 0.4511 - acc: 0.855 - ETA: 0s - loss: 0.4406 - acc: 0.860 - ETA: 0s - loss: 0.4372 - acc: 0.858 - ETA: 0s - loss: 0.4302 - acc: 0.860 - ETA: 0s - loss: 0.4339 - acc: 0.859 - ETA: 0s - loss: 0.4314 - acc: 0.859 - ETA: 0s - loss: 0.4315 - acc: 0.862 - ETA: 0s - loss: 0.4300 - acc: 0.862 - ETA: 0s - loss: 0.4333 - acc: 0.861 - ETA: 0s - loss: 0.4347 - acc: 0.861 - ETA: 0s - loss: 0.4331 - acc: 0.862 - ETA: 0s - loss: 0.4307 - acc: 0.862 - ETA: 0s - loss: 0.4330 - acc: 0.862 - 1s 144us/step - loss: 0.4289 - acc: 0.8638 - val_loss: 1.4682 - val_acc: 0.6320

Epoch 00048: val_loss did not improve from 1.16043
Epoch 49/50
7895/7895 [==

In [12]:
model.summary()

# Calculate training accuracy 
score_train = model.evaluate(x_train, y_train, verbose=1)
score_test = model.evaluate(x_test, y_test, verbose=1)
accuracy_train = 100*score_train[1]
accuracy_test = 100*score_test[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               51456     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
Total para

In [13]:
print("Training accuracy: %.4f%%" % accuracy_train)
print("Test accuracy: %.4f%%" % accuracy_test)

Training accuracy: 94.9335%
Test accuracy: 64.8746%


In [72]:
def extract_features(file_path):
    y,sr=librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(1,200))
    #print(features.shape)
    return features

In [73]:
class_lable=["air_conditioner","car_horn","children_playing","dog_bark","drilling","engine_idling","gun_shot","jackhammer","siren","street_music"]
def print_prediction(file_path):
    features = extract_features(file_path)
    predicted_vector = model.predict_classes(features)
    predicted_class = class_lable[predicted_vector[0]] 
    print("The predicted class is:", predicted_class, '\n') 

In [69]:
file_path='UrbanSound8K/audio/fold5/100032-3-0-0.wav'
print_prediction(file_path)

(1, 200)
The predicted class is: dog_bark 



In [74]:
file_path='UrbanSound8K/audio/fold7/101848-9-0-0.wav'
print_prediction(file_path)

The predicted class is: street_music 



In [75]:
file_path='UrbanSound8K/audio/fold10/200460-6-1-0.wav'
print_prediction(file_path)

The predicted class is: gun_shot 



In [76]:
file_path='Sample Auido/Car_horn.wav'
print_prediction(file_path)

The predicted class is: street_music 



In [77]:
file_path='Sample Auido/dog_bark.wav'
print_prediction(file_path)

The predicted class is: dog_bark 



In [79]:
file_path='Sample Auido/drilling.wav'
print_prediction(file_path)

The predicted class is: drilling 



In [80]:
file_path='Sample Auido/engine.wav'
print_prediction(file_path)

The predicted class is: engine_idling 

